# Bradford 0-19 Children and Young Peoples' Outcomes Framework: Regression Analysis

In [ ]:
import os
import contextlib

with open(os.devnull, "w") as f, contextlib.redirect_stderr(f):
    import rpy2.robjects as ro

%load_ext rpy2.ipython

In [ ]:
import pandas as pd
from rich import print
from IPython.display import display, Image
from sqlalchemy import create_engine
from importlib import reload
from pathlib import Path

import plotly.io as pio
pio.renderers.default = "notebook_connected"

from utils import (
    show_tab_model,
    make_crosstab
)

import warnings
warnings.filterwarnings('ignore')

# Load data

In [ ]:
#| echo: false

tbl_name = 'person_linked_2016plus'

# Note: The connection string is specific to the Connected Bradford VDE.
# Replace the placeholder below with the internal server URL.
conn_str = "DATABASE_URL_PLACEHOLDER"

# Create SQLAlchemy engine
engine = create_engine(conn_str)

df = pd.read_sql(f"SELECT * FROM [dbo].[{tbl_name}_derived]", engine)

print(df.shape)

In [ ]:
year_min = df.birth_datetime.min()
year_max = df.birth_datetime.max()

print(f"DOB covers for the dataset is from {year_min} to {year_max}.")

# Set up

In [ ]:
df.drop(columns=["FSP_GLD"], inplace=True)

df.rename(columns={
    'Valid_2y_HV': "Has_HV",
    "Valid_2y_ASQ": "Has_ASQ",
    "ASQ_DomainBinary_FGLD": "ASQ_FGLD_dom",
    "FSP_GLD_derived": "FSP_GLD",
    "ASQ_PHE_Risk": "ASQ_GLD"
}, inplace=True)


df['FSP_Present'] = df['FSP_Present'].map({True: 1, False: 0})

In [ ]:
%%R

options(warn = -1)

pkgs <- c(
  "dplyr", "sjPlot", "tidyr", "car", "forcats", "broom", 
  "interactions", "ggeffects", "ggplot2", "gtsummary", "lubridate",
  "gt", "webshot2", "Cairo", "performance", "labelled", "flextable",
  "officer", "magrittr", "gtExtras"
)

invisible(lapply(pkgs, function(p) {
  suppressPackageStartupMessages(
    library(p, character.only = TRUE, warn.conflicts = FALSE)
  )
}))

options(repr.plot.width = 10, repr.plot.height = 8, repr.plot.res = 300)
options(jupyter.plot_mimetypes = "image/svg+xml")
options(device = function(...) CairoPNG(...))

save_docs <- FALSE

results_dir <- file.path(getwd(), "Results")

if (!dir.exists(results_dir)) {
  dir.create(results_dir, recursive = TRUE)
}

labels_map <- list(
  gender       = "Sex",
  ethnicity    = "Ethnicity",      
  IMD          = "IMD (quintile)", 
  Has_HV       = "Has 2y HV",
  age_months   = "Age at FSP (months)", 
  ASQ_GLD      = "ASQ-3 GLD",
  ASQ_FGLD_dom = "ASQ-3 GLD (Domain)",
  ASQ_Composite = "ASQ-3 Composite Score"
)

prepare_df <- function(df,
                       factor_cols = c("gender", "ethnicity_group", "IMD19_quintile", "Has_HV", "Has_ASQ", "ASQ_FGLD_dom", "ASQ_GLD"), # just default setting
                       numeric_cols = NULL,
                       reference_levels = list(
                         ethnicity_group = "White British",
                         gender = "Female",
                         IMD19_quintile = "5",
                         Has_HV = "0",
                         Has_ASQ = "0",
                         ASQ_FGLD_dom = "0",
                         ASQ_GLD = "0"
                       )) {
  
  if ("gender" %in% names(df)) {
    df$gender[df$gender == "Unknown/Other"] <- NA
  }
  
  if (!is.null(numeric_cols)) {
    existing_num_cols <- numeric_cols[numeric_cols %in% names(df)]
    
    df[existing_num_cols] <- lapply(df[existing_num_cols], function(x) {
      suppressWarnings(as.numeric(as.character(x)))
    })
    
    factor_cols <- setdiff(factor_cols, existing_num_cols)
  }
  
  existing_factor_cols <- factor_cols[factor_cols %in% names(df)]
  df[existing_factor_cols] <- lapply(df[existing_factor_cols], factor)
  
  for (col in names(reference_levels)) {
    if (col %in% existing_factor_cols) {
      ref <- reference_levels[[col]]
      lev <- levels(df[[col]])
      if (!ref %in% lev) {
        warning(sprintf(
          "Reference '%s' is not a level of %s. Existing levels: %s",
          ref, col, paste(lev, collapse = ", ")
        ))
        next
      }
      df[[col]] <- relevel(df[[col]], ref = ref)
    }
  }

  cat("Reference levels used in this logit model:\n")
  invisible(
    sapply(existing_factor_cols, function(col) {
      cat(sprintf("  %s reference: %s\n", col, levels(df[[col]])[1]))
    })
  )
  cat("\n")
  
  rename_map <- list(
    ethnicity = "ethnicity_group",
    IMD = "IMD19_quintile",
    age_months = "age_fsp_months" 
   )

  for (new_name in names(rename_map)) {
    old_name <- rename_map[[new_name]]
    if (old_name %in% names(df)) {
      df <- df %>% rename(!!new_name := all_of(old_name))
    }
  }
  
  existing_labels <- labels_map[names(labels_map) %in% names(df)]

  df <- df %>% 
    set_variable_labels(.labels = existing_labels)
  
  return(df)
}
                       
droplevels_keep_labels <- function(df) {
  labels <- lapply(df, function(x) attr(x, "label"))

  df2 <- droplevels(df)

  for (nm in names(labels)) {
    if (!is.null(labels[[nm]])) {
      attr(df2[[nm]], "label") <- labels[[nm]]
    }
  }

  df2
}

get_perf <- function(x) {
  perf <- performance::model_performance(x) %>% as.data.frame()
  if ("n_Obs" %in% names(perf)) perf$nobs <- perf$n_Obs
  if (!"nobs" %in% names(perf)) perf$nobs <- stats::nobs(x)
  r2_val <- performance::r2(x)
  
  if (inherits(x, "glm")) {
    perf$R2_Tjur <- as.numeric(r2_val[[1]])
    perf$R2_adjusted <- NA
  } else {
    perf$R2_Tjur <- NA
    perf$R2_adjusted <- if("R2_adjusted" %in% names(perf)) perf$R2_adjusted else as.numeric(r2_val[[1]])
    perf$R2 <- if("R2" %in% names(perf)) perf$R2 else as.numeric(r2_val[[1]])
  }
  
  return(perf)
}

make_reg_tbl <- function(m, ref_symbol = "Reference") {
  is_glm <- inherits(m, "glm")

  tbl_regression(
    m,
    exponentiate = is_glm,
    conf.int = TRUE,
    conf.level = 0.95
    # add_estimate_to_reference_rows = TRUE
  ) %>%
    add_glance_table(
      glance_fun = get_perf,
      include = c("nobs", "R2_Tjur", "R2_adjusted"),
      label = list(
        nobs ~ "No. Obs.",
        R2_Tjur ~ "R² Tjur",
        R2_adjusted ~ "R² (Adj.)"
      )
    ) %>%
    modify_table_body(~ .x %>%
      dplyr::filter(
        !(row_type == "glance_statistic" &
          dplyr::if_all(
            tidyselect::where(is.numeric),
            ~ is.na(.x)
          ))
      )
    ) %>%
    modify_table_styling(
      # columns = label,
      columns = everything(),
      rows = row_type == "glance_statistic",
      text_format = "bold"
    ) %>%
    bold_labels() %>%
    bold_p() %>%
    modify_fmt_fun(
      p.value = function(x) style_pvalue(x, digits = 3)
    ) %>%
    modify_header(
      label    ~ "**Predictors**",
      estimate ~ ifelse(is_glm, "**OR (95% CI)**", "**$\\beta$ (95% CI)**"),
      p.value  ~ "***p***"
    ) %>%
    modify_table_styling(
      columns = c(estimate, ci),
      rows = row_type %in% c("label", "level") & !dplyr::coalesce(reference_row, FALSE) &
         !is.na(ci) & ci != "",
      cols_merge_pattern = "{estimate} ({conf.low} - {conf.high})"
    ) %>%
    modify_table_styling(columns = ci, hide = TRUE) %>%
    modify_table_styling(
      columns = estimate,
      rows = reference_row,
      text_format = "italic",
      missing_symbol = ref_symbol,
      align = "center"
    )
}


pick_mid_model <- function(models) {
  n <- length(models)
  if (n == 0) return(NA_integer_)

  mid <- (n + 1) / 2

  cand <- which(vapply(
    models,
    function(m) length(stats::coef(m)) - 1 > 1,
    logical(1)
  ))

  if (length(cand) == 0) return(NA_integer_)

  cand[which.min(abs(cand - mid))]
}


make_merged_table <- function(models,
                              dv_labels,
                              sort_predictors = TRUE) {
  # bold spanners
  dv_labels <- paste0("**", dv_labels, "**")

  # choose which table shows "Reference"
  fill_idx <- pick_mid_model(models)

  # build individual tables
  tbl_list <- vector("list", length(models))
  for (i in seq_along(models)) {
    ref_symbol <- if (!is.na(fill_idx) && i == fill_idx) "Reference" else ""
    tbl_list[[i]] <- make_reg_tbl(models[[i]], ref_symbol = ref_symbol)
  }

  # merge
  out <- gtsummary::tbl_merge(tbl_list, tab_spanner = dv_labels)

  # optional sorting
  if (isTRUE(sort_predictors)) {
  asq_vars <- c("ASQ-3")

  out <- out %>%
    gtsummary::modify_table_body(~ .x %>%
      dplyr::mutate(.orig_order = dplyr::row_number()) %>%
      dplyr::arrange(
        row_type == "glance_statistic",  
        dplyr::if_else(
          row_type == "glance_statistic",
          .orig_order, 
          dplyr::case_when(
            grepl("^ASQ", variable) ~ 0,                                
            TRUE ~ as.numeric(factor(variable)) + 1                   
          )
        )
      ) %>%
      dplyr::select(-.orig_order)
    )
}
  out
}
                              
make_gt <- function(tbl, padding_left_px = 50, label_col = "label") {
  tbl %>%
    as_gt() %>%
    gt::tab_style(
      style = gt::cell_text(indent = gt::px(padding_left_px)),
      locations = list(
        gt::cells_body(columns = -gt::all_of(label_col)),
        gt::cells_column_labels(columns = -gt::all_of(label_col))
      )
    )
}

make_outputs <- function(results_dir, name) {
  list(
    html = file.path(results_dir, paste0(name, "_fig.html")),
    rds  = file.path(results_dir, paste0(name, "_model.rds")),
    rds_gt = file.path(results_dir, paste0(name, "_fig_gt.rds")),
    docx = file.path(results_dir, paste0(name, "_fig.docx"))
  )
}

safe_docx <- function(gt_tbl, filename) {
  tryCatch(
    {
      gt::gtsave(gt_tbl, filename = filename)
      message("DOCX saved: ", filename)
      TRUE
    },
    error = function(e) {
      message("DOCX NOT saved (ignored): ", filename)
      message("Reason: ", conditionMessage(e))
      FALSE
    }
  )
}

## Create person table

In [ ]:
p_cols = ['ethnicity_group', 'gender', 'IMD19_deciles', 'IMD19_quintile', 'age_2_5', 'gender_raw', 'birth_datetime', 'FSP_ACADYR']

derived_cols = ['Has_HV', 'Has_ASQ', 'FSP_GLD', 'FSP_TotalScore', 'ASQ_FGLD', 'ASQ_GLD', 'ASQ_Composite', 'age_fsp_months', 'FSP_Present', "ASQ_Status", 'ASQ_Version', 'ASQ_n_domains']

df_person = df[['person_id'] + p_cols + derived_cols].drop_duplicates()

df_person = (
    df_person
    .sort_values(
        by=['Has_ASQ', 'ASQ_Version'],
        ascending=[False, False]   # priority: Yes + latest version
    )
    .drop_duplicates(subset='person_id', keep='first')
)

assert df_person['person_id'].is_unique, "person_id is not unique in df_person"

df_person_domains = pd.merge(
    df_person,
    df[['person_id', 'ASQ_Version', 'ASQ_Domain', 'ASQ_FGLD_dom', 'Has_ASQ'] + [c for c in df.columns if c.startswith('FSP_') and '_Binary' in c]].drop_duplicates(),
    on=['person_id', 'ASQ_Version', 'Has_ASQ'],
    how='left'
)

def check_conflict(df, col):
    bad = df.groupby("person_id")[col].nunique(dropna=True)
    conflict_ids = bad[bad > 1]
    if len(conflict_ids) > 0:
        print(f"⚠️ {col} has {len(conflict_ids)} conflicting persons")
        return df[df.person_id.isin(conflict_ids.index)][['person_id', col]].sort_values('person_id')
    # else:
    #     print(f"✓ {col} OK")

for col in p_cols + derived_cols:
    check_conflict(df_person, col)

# Sociodemographic characteristics of children in the analysis

In [ ]:
%%R -i df_person -o output_tbl_html

out <- make_outputs(results_dir, "tbl_demo_asq_eyfsp")

output_tbl_html <- out$html

df0 <- df_person %>%
  mutate(
    gender = factor(gender),
    ethnicity_group = factor(ethnicity_group),
    IMD19_quintile = factor(IMD19_quintile),
    # IMD19_quintile = fct_explicit_na(factor(IMD19_quintile), na_level = "Unknown")
  )


df_asq <- df0 %>%
  filter(
    !is.na(person_id),

    Has_ASQ == 1,
    ASQ_n_domains == 5,
    !is.na(ASQ_Composite),
  )
  
df_eyfsp <- df0 %>%
  filter(
    !is.na(person_id),

    # EYFSP
    (!is.na(FSP_GLD))
  )

df_eyfsp_linked <- df0 %>%
  filter(
    !is.na(person_id),

    # EYFSP
    (!is.na(FSP_GLD)),
    
    # linked ASQ complete
    Has_ASQ == 1,
    ASQ_n_domains == 5,
    !is.na(ASQ_Composite),
  )

vars_tbl1 <- c("gender", "ethnicity_group", "IMD19_quintile")

tbl1_labels <- list(
  gender ~ "Sex",
  ethnicity_group ~ "Ethnicity",
  IMD19_quintile ~ "IMD 2019 Quintile"
)

tbl1_stat <- list(
  all_categorical() ~ "{n} ({p}%)",
  all_continuous()  ~ "{mean} ({sd})"
)

make_tbl <- function(data, vars = vars_tbl1) {
  data %>%
    select(all_of(vars)) %>%
    tbl_summary(
      statistic = tbl1_stat,
      digits = list(
          all_categorical() ~ c(0, 1)
      ),
      missing = "ifany",
      missing_text = "Unknown",
      missing_stat = "{N_miss} ({p_miss}%)",
      label = tbl1_labels
    )
}

tbl_asq          <- make_tbl(df_asq)
tbl_eyfsp        <- make_tbl(df_eyfsp)
tbl_eyfsp_linked <- make_tbl(df_eyfsp_linked)

table1 <- tbl_merge(
  tbls = list(tbl_asq, tbl_eyfsp, tbl_eyfsp_linked),
  tab_spanner = c(
    "\u2003\u2003\u2003**ASQ-3**\u2003\u2003\u2003",
    "\u2003\u2003\u2003**EYFSP**\u2003\u2003\u2003",
    "\u2003**EYFSP + ASQ-3**\u2003"
  )
) %>%
  # modify_caption("**Table 1. Sociodemographic characteristics of children in the ASQ-3 and EYFSP analyses**") %>%
  bold_labels() %>%
  modify_header(label ~ "**Predictors**")

gt_tbl <- as_gt(table1)

gt::gtsave(gt_tbl, output_tbl_html)
# saveRDS(gt_tbl, out$rds_gt)
saveRDS(gt_tbl, out$rds)

In [ ]:
p = Path(output_tbl_html[0])
print(f"[green]Table saved to:[/] {p.parent.name}/{p.name}")

show_tab_model(output_tbl_html)

# The sociodemographic predictors of achieving a GLD on the ASQ-3

<details>
<summary style="background-color:#E6F4EA; padding:8px; border:1px solid #003366; border-radius:5px; cursor:pointer; font-weight:bold; color:#003366;">
ASQ-3 Scoring and Derivation of Composite Indicators (click to expand)
</summary>

The **Ages and Stages Questionnaire (ASQ-3)** includes three response options for each question:  

| Response   | Score |
|-------------|--------|
| Yes         | 10     |
| Sometimes   | 5      |
| Not yet     | 0      |

Each of the five developmental domains (**Communication**, **Gross Motor**, **Fine Motor**, **Problem Solving**, and **Personal-Social**) has a total possible score ranging from **0 to 60**.  

However, cut-offs differ by both age version (24, 27, or 30 months) and domain, meaning that raw scores are **not directly comparable** across all versions.

Therefore, it is not possible to derive a single continuous ASQ score with consistent meaning across ages.

<br>


**Domain-level classification**

Each domain score is categorised into one of three levels based on age-specific cut-offs:

| Category          | Interpretation                                                  |
|-------------------|-----------------------------------------------------------------|
| **Below Cut-Off** | Requires further assessment or professional intervention        |
| **Monitor**       | Development should be monitored                                 |
| **Above Cut-Off** | Development is on schedule                                      |


<br>


**Binary overall ASQ indicators**

Two binary indicators are derived to summarise each child’s overall developmental status, following two conventions:

**(1) PHE Convention (Public Health England definition)**  
- **At Risk (0):** If *any* of the five domains are classified as **Below Cut-Off**.  
- **Not at Risk (1):** If *all* domains are either **Monitor** or **Above Cut-Off**.

**(2) FGLD Convention (Full Good Level of Development)**  
- **Good Level of Development (1):** If *all* five domains are **Above Cut-Off** (“No Risk”).  
- **Not GLD (0):** If *any* domain is classified as **Monitor** or **Below Cut-Off**.


<br>

**Continuous ASQ composite score**

A continuous composite score is also calculated:

$\text{ASQ Composite Score} = \sum_{d=1}^{5} \text{DomainBinary}_d$

Each domain contributes:  
- 1 = Above Cut-Off (No Risk)  
- 0 = Monitor or Below Cut-Off  

This produces a total score ranging from **0 to 5**, where higher values indicate **fewer developmental concerns**.

<br>

**Notes**  
- Domain-level categories are derived using **age-appropriate validated cut-offs** for each ASQ version (24, 27, or 30 months).  

Below are the cut-offs used:

| ASQ_version | ASQ_Domain       | Below Cut-Off | Monitor Cut-Off | No Risk Min |
|--------------|------------------|--------------------|----------------|-------------|
| 30 | Communication    | 33 | 44 | 45 |
| 27 | Communication    | 24 | 36 | 37 |
| 24 | Communication    | 25 | 38 | 39 |
| 30 | Gross Motor      | 36 | 44 | 45 |
| 27 | Gross Motor      | 28 | 38 | 39 |
| 24 | Gross Motor      | 38 | 45 | 46 |
| 30 | Fine Motor       | 19 | 34 | 35 |
| 27 | Fine Motor       | 18 | 30 | 31 |
| 24 | Fine Motor       | 35 | 43 | 44 |
| 30 | Problem Solving  | 27 | 39 | 40 |
| 27 | Problem Solving  | 27 | 39 | 40 |
| 24 | Problem Solving  | 29 | 39 | 40 |
| 30 | Personal-Social  | 32 | 40 | 41 |
| 27 | Personal-Social  | 25 | 35 | 36 |
| 24 | Personal-Social  | 31 | 40 | 41 |

</details>

## Preparatory Work

### Create dataset of children with complete ASQ per domain (regardless of 2y HV status)

In [ ]:
df_person_asq = df_person[df_person['Has_ASQ'] == 1]

df_complete_asq = df_person_asq[df_person_asq['ASQ_n_domains'] == 5]

version_counts = (
    df_complete_asq.groupby('ASQ_Version')['person_id']
    .nunique()
    .sort_index()
)

version_summary = "\n".join(
    f"  - [bold]{ver}[/bold]: {n:,}" for ver, n in version_counts.items()
)

print(
    f"[bold cyan]Eligible sample:[/bold cyan] "
    f"{df_complete_asq.person_id.nunique():,} unique children with a valid ASQ completed at or before 2.5 years of age, "
    f"using the most recent ASQ record with complete data across all five developmental domains.\n"
    # f"[bold cyan]Breakdown by ASQ version:[/bold cyan]\n{version_summary}"
)

_ = make_crosstab(
    df_complete_asq,
    row_var="ASQ_Version",
    col_var='ASQ_Composite',
    caption_prefix="ASQ_Version x ASQ_Composite",
)

## Model results

### Overall - dichotomous outcome (GLD vs not GLD)

In [ ]:
df_tmp = df_complete_asq.copy()

In [ ]:
%%R -i df_tmp -o output_file_html

df_tmp <- prepare_df(df_tmp)

same_name <- "RQ3_1_bin"
out <- make_outputs(results_dir, same_name)
output_file_html <- out$html

model_rq31_bin <- glm(
  ASQ_GLD ~ gender + ethnicity + IMD,
  data = df_tmp,
  family = "binomial"
)

saveRDS(model_rq31_bin, file = out$rds)

tbl <- make_reg_tbl(model_rq31_bin) 

gt_tbl_rq31_bin <- make_gt(tbl)

gt::gtsave(
  gt_tbl_rq31_bin,
  filename = output_file_html
)

saveRDS(
  gt_tbl_rq31_bin,
  file = out$rds_gt
)

save_docs && safe_docx(gt_tbl_rq31_bin, out$docx)

In [ ]:
p = Path(output_file_html[0])
print(f"[green]Table saved to:[/] {p.parent.name}/{p.name}")

show_tab_model(output_file_html)

### Overall - continuous outcome

**Note**: 

A continuous composite score is also calculated:

$\text{ASQ Composite Score} = \sum_{d=1}^{5} \text{DomainBinary}_d$

Each domain contributes:  
- 1 = Above Cut-Off (No Risk)  
- 0 = Monitor or Below Cut-Off  

This produces a total score ranging from **0 to 5**, where higher values indicate **fewer developmental concerns**.

In [ ]:
%%R -i df_tmp -o output_file_html

df_tmp <- prepare_df(df_tmp)

same_name <- "RQ3_1_con"
out <- make_outputs(results_dir, same_name)
output_file_html <- out$html

model_rq31_con <- lm(
  ASQ_Composite ~ gender + ethnicity + IMD,
  data = df_tmp
)

saveRDS(model_rq31_con, file = out$rds)

tbl <- make_reg_tbl(model_rq31_con)

gt_tbl_rq31_con <- make_gt(tbl)

gt::gtsave(
  gt_tbl_rq31_con,
  filename = output_file_html
)

saveRDS(
  gt_tbl_rq31_con,
  file = out$rds_gt
)

save_docs && safe_docx(gt_tbl_rq31_con, out$docx)

In [ ]:
p = Path(output_file_html[0])
print(f"[green]Table saved to:[/] {p.parent.name}/{p.name}")

show_tab_model(output_file_html)

### Sensitivity analysis - IMD (binary outcome)

In [ ]:
%%R -i df_tmp -o output_file_html1 -o output_file_html2

df_tmp <- prepare_df(df_tmp)

# -------------------------
# Sensitivity 1: IMD missing as an explicit category
# -------------------------
same_name1 <- "RQ3_1_sens_IMD_missing_category"
out1 <- make_outputs(results_dir, same_name1)
output_file_html1 <- out1$html

df_s1 <- df_tmp
df_s1$IMD_misscat <- as.character(df_s1$IMD)
df_s1$IMD_misscat[is.na(df_s1$IMD_misscat) | df_s1$IMD_misscat == ""] <- "Missing"
df_s1$IMD_misscat <- factor(df_s1$IMD_misscat)

model1 <- glm(
  ASQ_GLD ~ gender + ethnicity + IMD_misscat,
  data = df_s1,
  family = "binomial"
)

saveRDS(model1, file = out1$rds)
# saveRDS(make_gt(make_reg_tbl(model1)), file = out1$rds_gt)
gt::gtsave(make_gt(make_reg_tbl(model1)), filename = output_file_html1)
save_docs && safe_docx(make_gt(make_reg_tbl(model1)), out1$docx)

# -------------------------
# Sensitivity 2: drop IMD (fit on full sample)
# -------------------------
same_name2 <- "RQ3_1_sens_no_IMD_fullsample"
out2 <- make_outputs(results_dir, same_name2)
output_file_html2 <- out2$html

model2 <- glm(
  ASQ_GLD ~ gender + ethnicity,
  data = df_tmp,
  family = "binomial"
)

saveRDS(model2, file = out2$rds)
gt::gtsave(make_gt(make_reg_tbl(model2)), filename = output_file_html2)
# saveRDS(make_gt(make_reg_tbl(model2)), file = out2$rds_gt)
save_docs && safe_docx(make_gt(make_reg_tbl(model2)), out2$docx)

#### Sensitivity 1: IMD missing as explicit category

In [ ]:
p = Path(output_file_html1[0])
print(f"[green]Table saved to:[/] {p.parent.name}/{p.name}")

show_tab_model(output_file_html1)

#### Sensitivity 2: drop IMD (fit on full sample)

In [ ]:
p = Path(output_file_html2[0])
print(f"[green]Table saved to:[/] {p.parent.name}/{p.name}")

show_tab_model(output_file_html2)

In [ ]:
del df_tmp

### By domain

In [ ]:
df_person_asq_gld_domain = df_person_domains[(df_person_domains['ASQ_n_domains'] == 5)].drop_duplicates()

df_tmp = df_person_asq_gld_domain.copy()

In [ ]:
%%R -i df_tmp -o output_file_html

same_name <- "RQ3_2"
out <- make_outputs(results_dir, same_name)
output_file_html <- out$html

domains <- unique(df_tmp$ASQ_Domain)
domains <- as.character(domains) 

df_tmp <- prepare_df(df_tmp, numeric_cols=c("ASQ_FGLD_dom"))

models_rq32 <- list()
tbls_rq32   <- list()

for (d in domains) {
    df_sub <- df_tmp %>%
        filter(ASQ_Domain == d) %>%
        droplevels() 

    models_rq32[[d]] <- glm(
        ASQ_FGLD_dom ~ gender + ethnicity + IMD,
        data = df_sub,
        family = "binomial"
    )

    tbls_rq32[[d]] <- make_reg_tbl(models_rq32[[d]])

}

if (length(models_rq32) == 0) {
    cat("\nNo valid models could be fitted. All domains were skipped.\n")
    output_file <- ""
} else {
  
    saveRDS(models_rq32, file = out$rds)
    
    # tbl_merged <- tbl_merge(
    #   tbls = tbls_rq32,
    #   tab_spanner = paste0("**ASQ ", names(tbls_rq32), "**")
    # )

    tbl_merged <- make_merged_table(models_rq32, paste0("ASQ ", names(tbls_rq32)), sort_predictors = TRUE)
    gt_tbl_rq32 <- make_gt(tbl_merged)

    gt::gtsave(gt_tbl_rq32, filename = output_file_html)
    saveRDS(gt_tbl_rq32, file = out$rds_gt)
    save_docs && safe_docx(gt_tbl_rq32, out$docx)
}

In [ ]:
p = Path(output_file_html[0])
print(f"[green]Table saved to:[/] {p.parent.name}/{p.name}")

show_tab_model(output_file_html)
del df_tmp

# The sociodemographic predictors of achieving a GLD on the EYFSP

For the **Early Years Foundation Stage Profile (EYFSP)**, outcomes will be analysed in two complementary formats:

- **Binary Good Level of Development (GLD) indicator:**  
  A binary variable will be derived following the national definition of GLD. Children will be classified as having a *Good Level of Development* if they achieve at least the expected level in all of the core areas: `communication and language`, `physical development`, `personal, social and emotional development`, `literacy`, and `mathematics`.

- **Continuous total EYFSP score:**  
  Where available, a continuous total score will also be calculated by summing the scores across all *Early Learning Goals (ELGs)*. This measure provides greater sensitivity for modelling and secondary analyses.

<br>

**Structure of the EYFSP assessment**

The EYFSP comprises **17 Early Learning Goals (ELGs)**, organised under seven areas of learning and development:

| Area of learning and development | Number of ELGs|
|----------------------------------|----------------|
| Communication and Language | 3  |
| Physical Development | 2 |
| Personal, Social and Emotional Development | 3 | 
| Literacy | 2 | 
| Mathematics | 2 | 
| Understanding the World | 3 | 
| Expressive Arts and Design | 2 | 

Each ELG is typically scored on a three-point scale:  **1 = Emerging**, **2 = Expected**, **3 = Exceeding**. A rating of **A** is recorded for children who have been granted an exemption from the assessment.

The total EYFSP score therefore ranges from **17 (all Emerging)** to **51 (all Exceeding)**.

## Preparatory Work

### Create dataset with valid EYFSP

In [ ]:
df_person_eyfsp = df_person[df_person['FSP_GLD'].notna()].drop_duplicates()
df_eyfsp = df.loc[df['FSP_GLD'].notna()]

print(f"[bold cyan]RQ4 eligible sample: {df_person_eyfsp.person_id.nunique():,} unique children who had a non-empty FSP_GLD entry (regardless of their HV and ASQ states).[/bold cyan]")
print(f"DOB range for RQ4 eligible sample: {df_person_eyfsp['birth_datetime'].min().date()} to {df_person_eyfsp['birth_datetime'].max().date()}. Academic years covered: {df_person_eyfsp['FSP_ACADYR'].min()} to {df_person_eyfsp['FSP_ACADYR'].max()}.")

today = pd.Timestamp.today().normalize()
df_person_eyfsp['age_5_0'] = df_person_eyfsp['birth_datetime'] + pd.DateOffset(days=1826)
# n_children_age_5 = df.loc[today >= df['age_5_0'], 'person_id'].nunique()
# print(f"Number of children aged 5 or older today in base table: {n_children_age_5:,}.")

## Model results

### Overall EYFSP GLD - dichotomous outcome (GLD vs not GLD)

In [ ]:
df_tmp = df_person_eyfsp.copy()

In [ ]:
%%R -i df_tmp -o output_file_html

same_name <- "RQ4_1_bin"
out <- make_outputs(results_dir, same_name)
output_file_html <- out$html

df_tmp <- prepare_df(df_tmp)

model_rq41_bin <- glm(
    FSP_GLD ~ gender + ethnicity + IMD,
    data = df_tmp,
    family = "binomial"
)

saveRDS(model_rq41_bin, file = out$rds)

tbl_rq41_bin <- make_reg_tbl(model_rq41_bin)

gt_tbl_rq41_bin <- as_gt(tbl_rq41_bin)

gt::gtsave(
  gt_tbl_rq41_bin,
  filename = output_file_html
)

saveRDS(
  gt_tbl_rq41_bin,
  file = out$rds_gt
)

save_docs && safe_docx(
  gt_tbl_rq41_bin,
  filename = out$docx
)

In [ ]:
p = Path(output_file_html[0])
print(f"[green]Table saved to:[/] {p.parent.name}/{p.name}")

show_tab_model(output_file_html)

### Overall EYFSP GLD - continuous outcome

In [ ]:
%%R -i df_tmp -o output_file_html

same_name <- "RQ4_1_con"
out <- make_outputs(results_dir, same_name)
output_file_html <- out$html

df_tmp <- prepare_df(df_tmp)

model_rq41_con <- lm(
    FSP_TotalScore ~ gender + ethnicity + IMD,
    data = df_tmp
)

saveRDS(model_rq41_con, file = out$rds)

tbl_rq41_con <- make_reg_tbl(model_rq41_con)

gt_tbl_rq41_con <- as_gt(tbl_rq41_con)

gt::gtsave(
  gt_tbl_rq41_con,
  filename = output_file_html
)

saveRDS(
  gt_tbl_rq41_con,
  file = out$rds_gt
)

save_docs && safe_docx(
  gt_tbl_rq41_con,
  filename = out$docx
)

In [ ]:
p = Path(output_file_html[0])
print(f"[green]Table saved to:[/] {p.parent.name}/{p.name}")

show_tab_model(output_file_html)

### Sensitivity analysis - IMD (binary outcome)

In [ ]:
%%R -i df_tmp -o output_file_html1 -o output_file_html2

# keep raw copy to protect the "full sample" sensitivity analysis
df_raw <- df_tmp

# -------------------------
# Sensitivity 1: IMD missing as explicit category
# -------------------------
same_name1 <- "RQ4_1_sens_IMD_missing_category"
out1 <- make_outputs(results_dir, same_name1)
output_file_html1 <- out1$html

df1 <- prepare_df(df_raw)

df1$IMD_misscat <- as.character(df1$IMD)
df1$IMD_misscat[is.na(df1$IMD_misscat) | df1$IMD_misscat == ""] <- "Missing"
df1$IMD_misscat <- factor(df1$IMD_misscat)

model1 <- glm(
  FSP_GLD ~ gender + ethnicity + IMD_misscat,
  data = df1,
  family = "binomial"
)

saveRDS(model1, file = out1$rds)
gt::gtsave(as_gt(make_reg_tbl(model1)), filename = output_file_html1)
# saveRDS(as_gt(make_reg_tbl(model1)), file = out1$rds_gt)
save_docs && safe_docx(as_gt(make_reg_tbl(model1)), filename = out1$docx)

# -------------------------
# Sensitivity 2: drop IMD (fit on full sample)
# -------------------------
same_name2 <- "RQ4_1_sens_no_IMD_fullsample"
out2 <- make_outputs(results_dir, same_name2)
output_file_html2 <- out2$html


# IMPORTANT: do NOT run prepare_df() if it drops rows due to IMD missing.
# If prepare_df() only does factor recoding and doesn't drop rows, you can use it.
# Safer default: only enforce factor levels needed for gender/ethnicity/outcome.
df2 <- df_raw
df2 <- prepare_df(df2, factor_cols = c("gender", "ethnicity", "FSP_GLD"), numeric_cols = NULL)

model2 <- glm(
  FSP_GLD ~ gender + ethnicity,
  data = df2,
  family = "binomial"
)

saveRDS(model2, file = out2$rds)
gt::gtsave(as_gt(make_reg_tbl(model2)), filename = output_file_html2)
# saveRDS(as_gt(make_reg_tbl(model2)), file = out2$rds_gt)
save_docs && safe_docx(as_gt(make_reg_tbl(model2)), filename = out2$docx)

#### Sensitivity 1: IMD missing as explicit category

In [ ]:
p = Path(output_file_html1[0])
print(f"[green]Table saved to:[/] {p.parent.name}/{p.name}")

show_tab_model(output_file_html1)

#### Sensitivity 2: drop IMD (fit on full sample)

In [ ]:
p = Path(output_file_html2[0])
print(f"[green]Table saved to:[/] {p.parent.name}/{p.name}")

show_tab_model(output_file_html2)

### Sensitivity analysis - Has ASQ (binary outcome)

In [ ]:
%%R -i df_tmp -o output_file_html1

# keep raw copy to protect the "full sample" sensitivity analysis
df_raw <- df_tmp

same_name1 <- "RQ4_1_sens_ASQ_1"
out1 <- make_outputs(results_dir, same_name1)
output_file_html1 <- out1$html

df1 <- prepare_df(df_raw)

model1 <- glm(
  FSP_GLD ~ Has_ASQ,
  data = df1,
  family = "binomial"
)

saveRDS(model1, file = out1$rds)
gt::gtsave(as_gt(make_reg_tbl(model1)), filename = output_file_html1)
# saveRDS(as_gt(make_reg_tbl(model1)), file = out1$rds_gt)
save_docs && safe_docx(as_gt(make_reg_tbl(model1)), filename = out1$docx)

In [ ]:
p = Path(output_file_html1[0])
print(f"[green]Table saved to:[/] {p.parent.name}/{p.name}")

show_tab_model(output_file_html1)

In [ ]:
%%R -i df_tmp -o output_file_html2

# keep raw copy to protect the "full sample" sensitivity analysis
df_raw <- df_tmp

same_name2 <- "RQ4_1_sens_ASQ_2"
out2 <- make_outputs(results_dir, same_name2)
output_file_html2 <- out2$html

df2 <- prepare_df(df_raw)

model2 <- glm(
  FSP_GLD ~ gender + ethnicity + IMD + Has_ASQ,
  data = df2,
  family = "binomial"
)

saveRDS(model2, file = out2$rds)
gt::gtsave(as_gt(make_reg_tbl(model2)), filename = output_file_html2)
# saveRDS(as_gt(make_reg_tbl(model2)), file = out2$rds_gt)
save_docs && safe_docx(as_gt(make_reg_tbl(model2)), filename = out2$docx)

In [ ]:
p = Path(output_file_html2[0])
print(f"[green]Table saved to:[/] {p.parent.name}/{p.name}")

show_tab_model(output_file_html2)

In [ ]:
del df_tmp

### Domain-level differences

In [ ]:
domain_binary_cols = [d for d in df.columns if d.startswith('FSP') and '_Binary' in d]
df_eyfsp_subset = df_eyfsp[['person_id', 'ethnicity_group', 'gender', 'FSP_GLD', 'IMD19_deciles', 'IMD19_quintile'] + domain_binary_cols].drop_duplicates()
core_for_gld = ['COM', 'PHY', 'PSE', 'LIT', 'MAT']

In [ ]:
df_tmp = df_eyfsp_subset.copy()

In [ ]:
%%R -i df_tmp -i core_for_gld -o output_file_html

same_name <- "RQ4_2"
out <- make_outputs(results_dir, same_name)
output_file_html <- out$html

df_tmp <- prepare_df(df_tmp)

# List of domains to model
domains <- core_for_gld

models_rq42 <- list()
tbls_rq42   <- list()

for (d in domains) {

    outcome_var <- paste0("FSP_", d, "_Binary")

    # subset domain rows that have valid 0/1
    df_sub <- df_tmp %>%
      filter(!is.na(.data[[outcome_var]])) %>%
      droplevels_keep_labels()

    cat("\nRunning model for", d, "domain...\n")

    # Fit logistic regression
    formula_str <- paste0(outcome_var, " ~ gender + ethnicity + IMD")
    models_rq42[[d]] <- glm(as.formula(formula_str), data = df_sub, family = "binomial")

    tbls_rq42[[d]] <- make_reg_tbl(models_rq42[[d]])
}

if (length(models_rq42) > 0) {
    saveRDS(models_rq42, file = out$rds)

    
    tbl_merged <- make_merged_table(models_rq42, paste0("FSP ", names(models_rq42)), sort_predictors = TRUE)

    gt_tbl_rq42 <- make_gt(tbl_merged)

    gt::gtsave(
      gt_tbl_rq42,
      filename = output_file_html
    )
    
    saveRDS(
      gt_tbl_rq42,
      file = out$rds_gt
    )
    
    save_docs && safe_docx(
      gt_tbl_rq42,
      filename = out$docx
    )
    
} else {
    output_file_html <- ""
}

In [ ]:
p = Path(output_file_html[0])
print(f"[green]Table saved to:[/] {p.parent.name}/{p.name}")

show_tab_model(output_file_html)
del df_tmp

# Do the developmental outcome on the ASQ-3 aged 2-years predict the developmental outcome on the EYFSP aged 5-years, using both dichotomous GLD and continuous score outcomes.  

In [ ]:
df_fsp_asq = df_person_domains[(df_person_domains['FSP_GLD'].notna()) & (df_person_domains['ASQ_GLD'].notna())].drop_duplicates()
df_person_fsp_asq = df_fsp_asq[['person_id', 'gender', 'ethnicity_group', 'ASQ_Composite', 'ASQ_FGLD', 'ASQ_GLD', 'FSP_TotalScore', 'FSP_GLD', 'age_fsp_months', 'IMD19_deciles', 'IMD19_quintile']].drop_duplicates()

print(f"[bold cyan]RQ5 eligible sample: {df_person_fsp_asq.person_id.nunique():,} unique children who had a non-empty FSP_GLD and complete ASQ entries.[/bold cyan]")

## Prediction of EYFSP total score at age 5 from ASQ-3 outcomes at age 2

In [ ]:
df_tmp = df_person_fsp_asq.copy()

In [ ]:
%%R -i df_tmp -o output_file_html

same_name <- "RQ5_1"
out <- make_outputs(results_dir, same_name)
output_file_html <- out$html

df_tmp <- prepare_df(df_tmp, numeric_cols=c("age_fsp_months", "ASQ_Composite", "ASQ_GLD"))

model_lm1 <- lm(
  FSP_TotalScore ~ ASQ_Composite,
  data = df_tmp
)

model_lm2 <- lm(
  FSP_TotalScore ~ ASQ_Composite + gender + ethnicity + age_months + IMD,
  data = df_tmp
)

model_lm3 <- lm(
  FSP_TotalScore ~ ASQ_GLD,
  data = df_tmp
)

model_lm4 <- lm(
  FSP_TotalScore ~ ASQ_GLD + gender + ethnicity + age_months + IMD,
  data = df_tmp
)

models_rq51 <- list(model_lm1, model_lm2, model_lm3, model_lm4)

saveRDS(models_rq51, file = out$rds)

dv_labels <- c(
  "EYFSP: Total Score by ASQ-3 Composite",
  "EYFSP: Total Score by ASQ-3 Composite (Adj.)",
  "EYFSP: Total Score by ASQ-3 GLD status",
  "EYFSP: Total Score by ASQ-3 GLD status (Adj.)"
)

dv_labels <- paste0("**", dv_labels, "**")

# final_table <- tbl_stack(tbl_list)

final_table <- make_merged_table(models_rq51, dv_labels, sort_predictors = TRUE)

gt_final_table <- make_gt(final_table)
  
gt::gtsave(gt_final_table, filename = output_file_html)
saveRDS(gt_final_table, file = out$rds_gt)
save_docs && safe_docx(gt_final_table, filename = out$docx)

In [ ]:
p = Path(output_file_html[0])
print(f"[green]Table saved to:[/] {p.parent.name}/{p.name}")

show_tab_model(output_file_html)

## Prediction of EYFSP GLD at age 5 from ASQ-3 outcomes at age 2

In [ ]:
%%R -i df_tmp -o output_file_html

same_name <- "RQ5_2"
out <- make_outputs(results_dir, same_name)
output_file_html <- out$html

df_tmp <- prepare_df(df_tmp, numeric_cols=c("age_fsp_months", "ASQ_Composite", "ASQ_GLD"))

model_lm1 <- glm(
  FSP_GLD ~ ASQ_Composite,
  data = df_tmp,
  family = "binomial"
)

model_lm2 <- glm(
  FSP_GLD ~ ASQ_Composite + gender + ethnicity + age_months + IMD,
  data = df_tmp,
  family = "binomial"
)

model_glm1 <- glm(
  FSP_GLD ~ ASQ_GLD,
  data = df_tmp,
  family = "binomial"
)

model_glm2 <- glm(
  FSP_GLD ~ ASQ_GLD + gender + ethnicity + age_months + IMD,
  data = df_tmp,
  family = "binomial"
)

models_rq52 <- list(model_lm1, model_lm2, model_glm1, model_glm2)
saveRDS(models_rq52, file = out$rds)

dv_labels <- c(
  "EYFSP: Odds of achieving GLD by ASQ-3 composite score (unadjusted)",
  "EYFSP: Odds of achieving GLD by ASQ-3 composite score (adjusted)",
  "EYFSP: Odds of achieving GLD by ASQ-3 GLD status (unadjusted)",
  "EYFSP: Odds of achieving GLD by ASQ-3 GLD status (adjusted)"
)

dv_labels <- paste0("**", dv_labels, "**")

tbl_list <- list()

for (i in seq_along(models_rq52)) {
  tbl_list[[i]] <- make_reg_tbl(models_rq52[[i]])
}

final_table <- make_merged_table(models_rq52, dv_labels, sort_predictors = TRUE)
  
gt_final_table <- make_gt(final_table)

gt::gtsave(gt_final_table, filename = output_file_html)
saveRDS(gt_final_table, file = out$rds_gt)
save_docs && safe_docx(gt_final_table, filename = out$docx)

In [ ]:
p = Path(output_file_html[0])
print(f"[green]Table saved to:[/] {p.parent.name}/{p.name}")

show_tab_model(output_file_html)
del df_tmp

## Do different developmental domain scores on the ASQ predict child development scores (EYFSP) at age 5

•	ASQ: Fine motor / gross motor – EYFSP: physical development

•	ASQ: Communication / Language - EYFSP: Communication

•	ASQ: Personal & Social – EYFSP: Personal, Social & emotional

•	ASQ: Problem solving – EYFSP: Problem solving


In [ ]:
df_person_fsp_asq = df_fsp_asq[
    ['person_id', 'gender', 'ethnicity_group', 'age_fsp_months', 'IMD19_deciles', 'IMD19_quintile',
     'ASQ_Domain', 'ASQ_FGLD_dom', 'ASQ_Composite', 'ASQ_FGLD', 'ASQ_GLD',
     'FSP_TotalScore', 'FSP_GLD']
    + [c for c in df_fsp_asq.columns if c.startswith('FSP_') and '_Binary' in c]
].drop_duplicates()

In [ ]:
df_tmp = df_person_fsp_asq.copy()

In [ ]:
%%R -i df_tmp -o output_file_html

same_name <- "RQ5_3_1"
out <- make_outputs(results_dir, same_name)
output_file_html <- out$html

df_tmp <- prepare_df(df_tmp, numeric_cols=c("age_fsp_months"))

# ----- Domain Mapping -----
# ASQ Domain to FSP Binary outcome mapping
domain_map <- list(
  "Fine Motor" = "FSP_PHY_Binary",
  "Gross Motor" = "FSP_PHY_Binary",
  "Communication" = "FSP_COM_Binary",
  "Personal-Social" = "FSP_PSE_Binary",
  "Problem Solving" = "FSP_MAT_Binary"
)

asq_domains <- intersect(names(domain_map), unique(df_tmp$ASQ_Domain))

models_rq53 <- list()

for (d in asq_domains) {
  target_col <- domain_map[[d]]
  df_sub <- df_tmp %>%
    filter(ASQ_Domain == d) %>%
    drop_na(ASQ_FGLD_dom, !!sym(target_col))
  
  if (nrow(df_sub) < 5) next
  
  m <- glm(formula = as.formula(paste0(target_col, " ~ ASQ_FGLD_dom")),
           data = df_sub, family = "binomial")
  models_rq53[[d]] <- m
}

saveRDS(models_rq53, file = out$rds)

dv_labels <- c(
  "FSP Physical: Odds by ASQ Fine Motor",
  "FSP Physical: Odds by ASQ Gross Motor",
  "FSP Comm: Odds by ASQ Communication",
  "FSP PSE: Odds by ASQ Personal-Social",
  "FSP Maths: Odds by ASQ Problem Solving"
)
# dv_labels <- paste0("**", dv_labels, "**")

tbl_list <- list()
for (i in seq_along(models_rq53)) {  
  tbl_list[[i]] <- make_reg_tbl(models_rq53[[i]])
}

final_table <- tbl_merge(
    tbl_list, 
    tab_spanner = paste0("**", dv_labels, "**")
  )

gt_final_table <- make_gt(final_table)

gt::gtsave(gt_final_table, filename = output_file_html)
saveRDS(gt_final_table, file = out$rds_gt)
save_docs && safe_docx(gt_final_table, filename = out$docx)

In [ ]:
p = Path(output_file_html[0])
print(f"[green]Table saved to:[/] {p.parent.name}/{p.name}")

show_tab_model(output_file_html)

In [ ]:
df_tmp = df_person_fsp_asq.copy()

In [ ]:
%%R -i df_tmp -o output_file_html

same_name <- "RQ5_3_2"
out <- make_outputs(results_dir, same_name)
output_file_html <- out$html

df_tmp <- prepare_df(df_tmp, numeric_cols=c("age_fsp_months"))

# ----- Domain Mapping -----
# ASQ Domain to FSP Binary outcome mapping
domain_map <- list(
  "Fine Motor" = "FSP_PHY_Binary",
  "Gross Motor" = "FSP_PHY_Binary",
  "Communication" = "FSP_COM_Binary",
  "Personal-Social" = "FSP_PSE_Binary",
  "Problem Solving" = "FSP_MAT_Binary"
)

asq_domains <- intersect(names(domain_map), unique(df_tmp$ASQ_Domain))

models_rq53 <- list()

for (d in asq_domains) {
  target_col <- domain_map[[d]]
  df_sub <- df_tmp %>%
    filter(ASQ_Domain == d) %>%
    drop_na(ASQ_FGLD_dom, !!sym(target_col))
  
  if (nrow(df_sub) < 5) next
  
  m <- glm(formula = as.formula(paste0(target_col, " ~ ASQ_FGLD_dom + gender + ethnicity + IMD")),
           data = df_sub, family = "binomial")
  models_rq53[[d]] <- m
}

saveRDS(models_rq53, file = out$rds)

dv_labels <- c(
  "FSP Physical: Odds by ASQ Fine Motor",
  "FSP Physical: Odds by ASQ Gross Motor",
  "FSP Comm: Odds by ASQ Communication",
  "FSP PSE: Odds by ASQ Personal-Social",
  "FSP Maths: Odds by ASQ Problem Solving"
)
# dv_labels <- paste0("**", dv_labels, "**")

tbl_list <- list()
for (i in seq_along(models_rq53)) {  
  tbl_list[[i]] <- make_reg_tbl(models_rq53[[i]])
}

final_table <- tbl_merge(
    tbl_list, 
    tab_spanner = paste0("**", dv_labels, "**")
  )

gt_final_table <- make_gt(final_table)

gt::gtsave(gt_final_table, filename = output_file_html)
saveRDS(gt_final_table, file = out$rds_gt)
save_docs && safe_docx(gt_final_table, filename = out$docx)

In [ ]:
p = Path(output_file_html[0])
print(f"[green]Table saved to:[/] {p.parent.name}/{p.name}")

show_tab_model(output_file_html)

## The association between ASQ-3 GLD at age 2 and EYFSP GLD at age 5 vary by socio-demographic characteristics.

In [ ]:
%%R -i df_tmp -o output_file_html

same_name <- "RQ5_4"
out <- make_outputs(results_dir, same_name)
output_file_html <- out$html

df_tmp <- prepare_df(df_tmp, numeric_cols=c("age_fsp_months"))


# --------------------------
# 1. Main effect model
# --------------------------
model_main <- glm(
  FSP_GLD ~ ASQ_GLD + gender + ethnicity + age_months + IMD,
  data = df_tmp,
  family = "binomial"
)

# --------------------------
# 2. ASQ × Gender interaction
# --------------------------
model_gender_int <- glm(
  FSP_GLD ~ ASQ_GLD * gender + ethnicity + age_months + IMD,
  data = df_tmp,
  family = "binomial"
)

# --------------------------
# 3. ASQ × Ethnicity interaction
# --------------------------
model_eth_int <- glm(
  FSP_GLD ~ ASQ_GLD * ethnicity + gender + age_months + IMD,
  data = df_tmp,
  family = "binomial"
)

# --------------------------
# 4. ASQ × IMD interaction
# --------------------------
model_imd_int <- glm(
  FSP_GLD ~ ASQ_GLD * IMD + gender + ethnicity + age_months,
  data = df_tmp,
  family = "binomial"
)

# --------------------------
# Output all models in one table
# --------------------------

models_rq54 <- list(
  model_main,
  model_gender_int,
  model_eth_int,
  model_imd_int
)

saveRDS(models_rq54, file = out$rds)

dv_labels <- c(
  "Main effect",
  "ASQ GLD x Gender",
  "ASQ GLD x Ethnicity",
  "ASQ GLD x IMD"
)

tbl_list <- list()

for (i in seq_along(models_rq54)) {  
  tbl_list[[i]] <- make_reg_tbl(models_rq54[[i]])
}

final_table <- make_merged_table(models_rq54, dv_labels, sort_predictors = TRUE)

gt_final_table <- make_gt(final_table)

gt::gtsave(gt_final_table, filename = output_file_html)
saveRDS(gt_final_table, file = out$rds_gt)
save_docs && safe_docx(gt_final_table, filename = out$docx)

In [ ]:
p = Path(output_file_html[0])
print(f"[green]Table saved to:[/] {p.parent.name}/{p.name}")

show_tab_model(output_file_html)
del df_tmp

## Summary of regression results

### Binary outcomes (GLD vs not GLD)

In [ ]:
%%R -o output_file_html

same_name <- "all_regression"
out <- make_outputs(results_dir, same_name)
output_file_html <- out$html

models_all <- list(
    "ASQ-3: Odds of achieving GLD" = model_rq31_bin,
    "EYFSP: Odds of achieving GLD" = model_rq41_bin,
    "EYFSP: Odds of achieving GLD by ASQ-3 GLD status" = models_rq52[[4]]
)

tbls_all <- list()

for (i in seq_along(models_all)) {  
  tbls_all[[i]] <- make_reg_tbl(models_all[[i]])
}

final_table <- make_merged_table(models_all, names(models_all), sort_predictors = TRUE)
  
gt_final_table <- make_gt(final_table)%>%
  tab_style(
    style = list(
      # cell_borders(sides = "right", color = "#D3D3D3", style = "dashed", weight = px(2)),
      "padding-right: 20px !important;"
    ),
    locations = cells_body(columns = c("p.value_1", "p.value_2"))
  ) %>%
  tab_style(
    style = "padding-left: 20px !important;",
    locations = cells_body(columns = c("estimate_2", "estimate_3"))
  )
  
saveRDS(gt_final_table, file = out$rds)
gt::gtsave(gt_final_table, filename = output_file_html)
saveRDS(gt_final_table, file = out$rds_gt)
save_docs && safe_docx(gt_final_table, filename = out$docx)

In [ ]:
p = Path(output_file_html[0])
print(f"[green]Table saved to:[/] {p.parent.name}/{p.name}")

show_tab_model(output_file_html)

### Continuous outcomes

In [ ]:
%%R -o output_file_html

same_name <- "all_regression_continuous"
out <- make_outputs(results_dir, same_name)
output_file_html <- out$html

models_all <- list(
    "ASQ-3: Composite Score by Demographics" = model_rq31_con,
    "EYFSP: Total Score by Demographics" = model_rq41_con,
    "EYFSP: Odds of achieving GLD by ASQ-3 Composite Score" = models_rq52[[2]]
)

tbls_all <- list()

for (i in seq_along(models_all)) {  
  tbls_all[[i]] <- make_reg_tbl(models_all[[i]])
}

final_table <- make_merged_table(models_all, names(models_all), sort_predictors = TRUE)
  
gt_final_table <- make_gt(final_table)%>%
  tab_style(
    style = list(
      # cell_borders(sides = "right", color = "#D3D3D3", style = "dashed", weight = px(2)),
      "padding-right: 20px !important;"
    ),
    locations = cells_body(columns = c("p.value_1", "p.value_2"))
  ) %>%
  tab_style(
    style = "padding-left: 20px !important;",
    locations = cells_body(columns = c("estimate_2", "estimate_3"))
  )
  
saveRDS(gt_final_table, file = out$rds)
gt::gtsave(gt_final_table, filename = output_file_html)
saveRDS(gt_final_table, file = out$rds_gt)
save_docs && safe_docx(gt_final_table, filename = out$docx)

In [ ]:
p = Path(output_file_html[0])
print(f"[green]Table saved to:[/] {p.parent.name}/{p.name}")

show_tab_model(output_file_html)

### Plots

In [ ]:
%%R

models_rq54 <- readRDS("./Results/RQ5_4_model.rds")
model_main <- models_rq54[[1]]
model_gender_int <- models_rq54[[2]]
model_eth_int <- models_rq54[[3]]
model_imd_int <- models_rq54[[4]]

plots_dir <- file.path(results_dir, "Plots_RQ5_4")
dir.create(plots_dir, showWarnings = FALSE, recursive = TRUE)

save_png <- function(p, filename, w = 10, h = 6, dpi = 300) {
  ggplot2::ggsave(
    filename = file.path(plots_dir, filename),
    plot = p,
    width = w, height = h, units = "in",
    dpi = dpi,
    bg = "white"
  )
}

# 1) Main and coefs plots
save_png(
  plot_model(
  model_main,
  type = "est",
  transform = "exp",        
  show.values = TRUE,
  value.size = 3.5,
  digits = 2,
  show.p = FALSE,          
  axis.title = c("Odds ratio (95% CI)", NULL)
) + theme_minimal(base_size = 14) +
    ggtitle("Main effects model"),
  "01_main_effect_plot_model.png"
)

save_png(
  plot_model(model_gender_int, type = "est", show.values = TRUE, value.size = 3) +
    ggtitle("ASQ x Gender") +
    theme_minimal(base_size = 14),
  "02_asq_x_gender_plot_model.png"
)

save_png(
  plot_model(model_eth_int, type = "est", show.values = TRUE, value.size = 3) +
    ggtitle("ASQ x Ethnicity") +
    theme_minimal(base_size = 14),
  "03_asq_x_ethnicity_plot_model.png"
)

save_png(
  plot_model(model_imd_int, type = "est", show.values = TRUE, value.size = 3) +
    ggtitle("ASQ x IMD") +
    theme_minimal(base_size = 14),
  "04_asq_x_imd_plot_model.png"
)

# 2) Interaction plots
save_png(
  cat_plot(
    model_gender_int,
    pred = ASQ_GLD,
    modx = gender,
    interval = TRUE,
    int.width = 0.95,
    plot.points = TRUE
  ) +
    labs(colour = "Gender", fill = "Gender") +
    ggtitle("Predicted probability by ASQ_GLD and Gender") +
    theme_minimal(base_size = 14),
  "05_cat_plot_asq_by_gender_ci.png"
)

save_png(
  cat_plot(
    model_eth_int,
    pred = ASQ_GLD,
    modx = ethnicity,
    interval = TRUE,
    int.width = 0.95,
    plot.points = TRUE
  ) +
    labs(colour = "Ethnicity", fill = "Ethnicity") +
    ggtitle("Predicted probability by ASQ_GLD and Ethnicity") +
    theme_minimal(base_size = 14),
  "06_cat_plot_asq_by_ethnicity_ci.png"
)

save_png(
  cat_plot(
    model_imd_int,
    pred = ASQ_GLD,
    modx = IMD,
    interval = TRUE,
    int.width = 0.95,
    plot.points = TRUE
  ) +
    labs(colour = "IMD (quintile)", fill = "IMD (quintile)") +
    ggtitle("Predicted probability by ASQ_GLD and IMD") +
    theme_minimal(base_size = 14),
  "07_cat_plot_asq_by_imd_ci.png"
)

# 3) ggeffects plots
save_png(
  ggpredict(model_gender_int, terms = c("ASQ_GLD", "gender"), ci_method = "wald") %>%
    plot() +
    ggtitle("ASQ_GLD by Gender") +
    theme_minimal(base_size = 14),
  "08_ggpredict_asq_by_gender_ci.png"
)

save_png(
  ggpredict(model_eth_int, terms = c("ASQ_GLD", "ethnicity"), ci_method = "wald") %>%
    plot() +
    ggtitle("ASQ_GLD by Ethnicity") +
    theme_minimal(base_size = 14),
  "09_ggpredict_asq_by_ethnicity_ci.png"
)

save_png(
  ggpredict(model_imd_int, terms = c("ASQ_GLD", "IMD"), ci_method = "wald") %>%
    plot() +
    ggtitle("ASQ_GLD by IMD") +
    theme_minimal(base_size = 14),
  "10_ggpredict_asq_by_imd_ci.png"
)

In [ ]:
from pathlib import Path

plots_dir = Path("Results") / "Plots_RQ5_4"
pngs = [str(p) for p in sorted(plots_dir.glob("*.png"))]

try:
    import ipyplot
    ipyplot.plot_images(pngs, img_width=800)
except:
    pass